In [ ]:
# Clone the mistral-finetune repo
!git clone https://github.com/mistralai/mistral-finetune.git
!pip install -r /content/mistral-finetune/requirements.txt
!pip install huggingface_hub

# Login to Huggingface
from huggingface_hub import notebook_login

notebook_login()

Cloning into 'mistral-finetune'...
remote: Enumerating objects: 439, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 439 (delta 140), reused 110 (delta 101), pack-reused 259
Receiving objects: 100% (439/439), 229.90 KiB | 22.99 MiB/s, done.
Resolving deltas: 100% (224/224), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-many

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

!mkdir -p /content/mistral_models
!cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
!rm -r /root/mistral_models/7B-v0.3

# Confirm the files have been downloaded
!ls /content/mistral_models/7B-v0.3

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

consolidated.safetensors  params.json  tokenizer.model.v3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import re
import json
import pandas as pd

file_path = "/content/drive/MyDrive/ACL/MultilingualLLMBias/training_data_bootstrap_mistral.txt"

# Function to parse the custom format
def parse_custom_format(line):
    match = re.search(r'\[INST\](.*?)\[\/INST\]\n```\npython\n(.*?)```</s>', line, re.DOTALL)
    if match:
        instruction = match.group(1).strip()
        code = match.group(2).strip()
        return {
            "messages": [
                {
                    "role": "user",
                    "content": instruction,
                },
                {
                    "role": "assistant",
                    "content": code,
                },
            ]
        }
    else:
        print(f"Line did not match the expected format: {line}")
        return None

# Function to load and parse the dataset
def load_custom_dataset(file_path):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Handle escape sequences
            line = bytes(line, "utf-8").decode("unicode_escape")
            parsed_line = parse_custom_format(line.strip())
            if parsed_line:
                dataset.append(parsed_line)
            else:
                print(f"Failed to parse line: {line.strip()}")
    return dataset

# Load and parse the dataset
data = load_custom_dataset(file_path)

# Print to confirm the data format
print(data)

# Split the dataset into training and evaluation sets
df = pd.DataFrame(data)
df_train = df.sample(frac=0.90, random_state=200)
df_eval = df.drop(df_train.index)

# Save to jsonl format
df_train.to_json("ultrachat_chunk_train_fixed.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval_fixed.jsonl", orient="records", lines=True)

print("Training and evaluation datasets have been saved.")


[{'messages': [{'role': 'user', 'content': '<<SYS>>\\nYou are an expert Python programmer, and here is your task:\\n<</SYS>>\\n\\nÐ\x92Ñ\x8b Ñ\x8fÐ²Ð»Ñ\x8fÐµÑ\x82ÐµÑ\x81Ñ\x8c Ð¾Ð¿Ñ\x8bÑ\x82Ð½Ñ\x8bÐ¼ Ð¿ÐµÑ\x80ÐµÐ²Ð¾Ð´Ñ\x87Ð¸ÐºÐ¾Ð¼, ÐºÐ¾Ñ\x82Ð¾Ñ\x80Ñ\x8bÐ¹ Ð¿ÐµÑ\x80ÐµÐ²Ð¾Ð´Ð¸Ñ\x82 Ñ\x84Ñ\x80Ð°Ð·Ñ\x8b Ñ\x81 Ð°Ð½Ð³Ð»Ð¸Ð¹Ñ\x81ÐºÐ¾Ð³Ð¾ Ñ\x8fÐ·Ñ\x8bÐºÐ° Ð½Ð° Ñ\x80Ð°Ð·Ð»Ð¸Ñ\x87Ð½Ñ\x8bÐµ Ð´Ñ\x80Ñ\x83Ð³Ð¸Ðµ Ñ\x8fÐ·Ñ\x8bÐºÐ¸. Ð\x9fÐµÑ\x80ÐµÐ²ÐµÐ´Ð¸Ñ\x82Ðµ Ñ\x81Ð»ÐµÐ´Ñ\x83Ñ\x8eÑ\x89Ð¸Ð¹ Ð°Ð½Ð³Ð»Ð¸Ð¹Ñ\x81ÐºÐ¸Ð¹ Ñ\x84Ñ\x80Ð°Ð·Ðµ Ð½Ð° Ñ\x80Ñ\x83Ñ\x81Ñ\x81ÐºÐ¸Ð¹:'}, {'role': 'assistant', 'content': 'def is_valid_tuple(tuple_element):\n    """\n    Check if given tuple is valid based on conditions.\n    """\n    if not isinstance(tuple_element[0], str) or tuple_element[0][0]!= \'b\':\n        return False'}]}, {'messages': [{'role': 'user', 'content': '<<SYS>>\\nYou are an expert Python programmer, and here is your task:\\n<</SYS>>\\n\\nÐ\x9dÐ°Ð¿Ð¸Ñ\x88Ð¸Ñ\x82Ðµ Ð¿Ñ\x80Ð¾Ð³Ñ\x80Ð°Ð¼Ð¼Ñ\x83 Ð½Ð° Python 

In [ ]:
!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py


!python reformat_data.py ultrachat_chunk_train_fixed.jsonl


!python reformat_data.py ultrachat_chunk_eval_fixed.jsonl


--2024-07-31 17:06:19--  https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3381 (3.3K) [text/plain]
Saving to: ‘reformat_data.py’

reformat_data.py    100%[===================>]   3.30K  --.-KB/s    in 0s      

2024-07-31 17:06:19 (64.0 MB/s) - ‘reformat_data.py’ saved [3381/3381]



In [ ]:
!mkdir -p /content/data

# Save the reformatted datasets to the /data directory
!mv ultrachat_chunk_train_fixed.jsonl /content/data/ultrachat_chunk_train_fixed.jsonl
!mv ultrachat_chunk_eval_fixed.jsonl /content/data/ultrachat_chunk_eval_fixed.jsonl

# Confirm the files exist
!ls /content/data


ultrachat_chunk_eval_fixed.jsonl  ultrachat_chunk_train_fixed.jsonl


In [ ]:

# Navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

# Validate and reformat the data
!python -m utils.reformat_data /content/data/ultrachat_chunk_train_fixed.jsonl
!python -m utils.reformat_data /content/data/ultrachat_chunk_eval_fixed.jsonl



/content/mistral-finetune


In [ ]:
import os
import yaml

# Set environment variables
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Define the training configuration
config = """
# data
data:
  instruct_data: "/content/data/ultrachat_chunk_train_fixed.jsonl"  # Ensure correct path
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/content/data/ultrachat_chunk_eval_fixed.jsonl"  # Ensure correct path

# model
model_id_or_path: "/content/mistral_models/7B-v0.3"  # Ensure correct path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 8192
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/content/test_ultra"  # Ensure correct path
"""

# Save the configuration to example.yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)



In [ ]:
# Ensure the run_dir has not been created before
!rm -r /content/test_ultra

# Start training
!torchrun --nproc-per-node 1 -m train example.yaml

rm: cannot remove '/content/test_ultra': No such file or directory
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2024-07-31 17:07:31.369659: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 17:07:31.387589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 17:07:31.408979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when

In [ ]:
# Copy the fine-tuned model to Google Drive
!cp -r /content/test_ultra /content/drive/MyDrive/ACL/Mistral-7b

In [ ]:
!pip install mistral_inference

In [ ]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

# Set the paths
tokenizer = MistralTokenizer.from_file("/content/mistral_models/7B-v0.3/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder("/content/mistral_models/7B-v0.3")  # change to extracted model dir
lora_path = "/content/drive/MyDrive/ACL/Mistral-7b/checkpoints/checkpoint_000100/consolidated/lora.safetensors"


# Prepare a completion request
completion_request = ChatCompletionRequest(messages=[UserMessage(content="Write a python function to remove second and last occurrence of a given character from the string.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)





def remove_first_last_occurrence(string, char):
    start_index = string.find(char)
    end_index = string.rfind(char)
    if start_index == -1 or end_index == -1:
        return string
